In [1]:
import nltk
import pandas as pd
import numpy as np


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jake/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from io import StringIO

col = ['author', 'text']
train = train[pd.notnull(train['text'])]


train['author_id'] = train['author'].factorize()[0]
author_id_df = train[['author', 'author_id']].drop_duplicates().sort_values('author_id')
author_to_id = dict(author_id_df.values)
id_to_category = dict(author_id_df[['author_id', 'author']].values)

author_id = train['author_id']
proper_noun_list  = []
proper_nouns = []
for sentence in train['text']:
    tokens = nltk.word_tokenize(sentence)
    pos = nltk.pos_tag(tokens)
    proper_noun = []
    for tag in pos:
        if (tag[1] == "NNP"):
            proper_noun_list.append(tag[0])
            proper_noun.append(tag[0])
    proper_nouns.append(proper_noun)
    
    
proper_noun_list = list(dict.fromkeys(proper_noun_list))
         
print(proper_noun_list)

['Windsor', 'Terrace', 'Superintendent', 'Epicurus', 'Greek', 'Orion', 'A', 'Herbert', 'West', 'Wheaton', 'Street', 'Draconian', 'Pestilence', 'Speculation', 'California', 'Von', 'Kempelen', 'Sarnath', 'Ib', 'Mnar', 'Paradise', 'Arnheim', 'Their', 'Sheehan', 'Old', 'Bugs', 'River', 'Innsmouth', 'D', 'Wilbur', 'Monday', 'Merrival', 'Underduk', 'Dr.', 'Johnson', 'Man', 'Aspect', 'Ellison', 'Nature', 'Mein', 'Gott', 'No', 'Fire', 'O', 'Raymond', 'Magazine', 'Constantinople', 'Greeks', 'G', 'Ahead', 'William', 'Latin', 'Cornelius', 'Agrippa', 'Albertus', 'Magnus', 'Paracelsus', 'Idris', 'Adrian', 'Legrand', 'Geneva', 'Mr.', 'Theodore', 'Sinivate', 'Gawd', 'Oh', 'Comte', 'Charles', 'Le', 'Sorcier', 'Perdita', 'Scotland', 'God', 'Ashimah', 'Main', 'Protectorate', 'Earl', 'Nith', 'Ingolstadt', 'Kidd', 'Look', 'Godhead', 'Galpin', 'New', 'York', 'University', 'English', 'Kilderry', 'Lofoden', 'Moskoe', 'Cumberland', 'Greece', 'Pierre', 'Moreau', 'Madame', "L'Espanaye", 'Hebrews', 'Heaven', 'Am

In [4]:
count = 0
for pn in proper_nouns:
    if(len(pn) != 0):
        count += 1

print(count/len(proper_nouns))
print(len(proper_noun_list))

0.36028397773124266
4084


In [5]:
def find_links(arr_nouns):
    nouns = []
    for noun1 in arr_nouns:
        for noun2 in arr_nouns:
            if(noun1 != noun2):
                nouns.append([noun1,noun2])
    return nouns


                
    
proper_nouns_linked = []
for pnn in proper_nouns:
    proper_nouns_linked.append(find_links(pnn))

    

In [6]:
eap_count = []
hpl_count = []
mws_count = []
links = []

for z in range(len(proper_noun_list)):
    noun = proper_noun_list[z]
    eap_count.append(0)
    hpl_count.append(0)
    mws_count.append(0)
    for i in range(len(author_id)):
        for noun2 in proper_nouns[i]:
            if(noun == noun2):
                if(author_id[i] == 0):
                    eap_count[z] += 1
                if(author_id[i] == 1):
                    hpl_count[z] += 1
                if(author_id[i] == 2):
                    mws_count[z] += 1
            

            
       

In [7]:
from collections import Counter

links = []
for z in range(len(proper_noun_list)):
    noun = proper_noun_list[z]
    cnt = Counter()
    for all_noun_links in proper_nouns_linked:
        for noun_links in all_noun_links:
            if(noun == noun_links[0]):
                cnt[noun_links[1]] += 1
    links.append(cnt)
          

In [8]:
df = pd.DataFrame(zip(proper_noun_list,eap_count,hpl_count,mws_count,links), columns = ['Noun', 'EAP', 'HPL', "MWS", "Links"]) 




In [9]:
df.loc['no']

,Noun,EAP,HPL,MWS,Links
0,Windsor,0,0,73,"{'Terrace': 1, 'Merrival': 1, 'Earl': 13, 'Cas..."
1,Terrace,0,0,2,"{'Windsor': 1, 'Castle': 1}"
2,Superintendent,0,5,0,{'Arthur': 1}
3,Epicurus,2,0,0,"{'Greek': 1, 'Orion': 2, 'Chantilly': 1, 'Dr.'..."
4,Greek,9,1,19,"{'Epicurus': 1, 'Orion': 1, 'Roman': 1, 'Cumbe..."
5,Orion,5,2,0,"{'Epicurus': 2, 'Greek': 1, 'Come': 1, 'Pleiad..."
6,A,27,3,4,"{'Smith': 14, 'General': 12, 'John': 13, 'B.':..."
7,Herbert,0,14,0,"{'West': 14, 'Boston': 2, 'Dr.': 3, 'Great': 1..."
8,West,3,105,0,"{'Herbert': 14, 'Boston': 3, 'Dr.': 9, 'Ottawa..."
9,Wheaton,0,1,0,{'Street': 1}
